In [1]:
import numpy as np
import sys
import scipy.sparse as sp
import torch
import torch.nn.functional as F
from data_loader import data_loader

In [2]:
prefix='iYO844'

dl = data_loader('../data/{}'.format(prefix))

In [3]:
dl.labels_test['data'][dl.labels_test['mask']]

array([ 3.40119738,  2.73631367, -4.82831374, -2.99573227, -0.14502577,
        0.60434583,  2.16332303, -0.26136476, -2.83021784, -1.1535649 ,
       -1.42711636,  3.97029191,  1.23837423, -3.72970145, -1.58032826,
        1.90210753,  2.41591378, -0.38566248])

In [4]:
dl.labels_test['data']

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  3.40119738,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  2.73631367,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [6]:
dl.nodes['attr'][0].keys()

dict_keys(['logits', 'single'])

In [12]:
features = []
for i in range(len(dl.nodes['count'])):
    th = dl.nodes['attr'][i]
    if th is None:
        features.append(sp.eye(dl.nodes['count'][i]))
    else:
        features.append(th)
        
print(features[0]['logits'].shape)
print(features[0]['single'].shape)
print(features[1].shape)

(384, 1600, 1600, 2)
(384,)
(616, 129)


In [23]:
adjM = sum(dl.links['data'].values())
# labels = np.zeros(
#     (dl.nodes['count'][0], dl.labels_train['num_labels']), dtype=float)

labels = np.zeros(
    (dl.nodes['count'][0],), dtype=float)

val_ratio = 0.2
train_idx = np.nonzero(dl.labels_train['mask'])[0]
np.random.shuffle(train_idx)
split = int(train_idx.shape[0]*val_ratio)
val_idx = train_idx[:split]
train_idx = train_idx[split:]
train_idx = np.sort(train_idx)
val_idx = np.sort(val_idx)
test_idx = np.nonzero(dl.labels_test['mask'])[0]

In [24]:
labels[train_idx].shape

(34,)

In [25]:
dl.labels_train['data'][train_idx].shape

(34,)

In [26]:
labels[train_idx] = dl.labels_train['data'][train_idx]
labels[val_idx] = dl.labels_train['data'][val_idx]
labels[test_idx] = dl.labels_test['data'][test_idx]

In [13]:
adjM = sum(dl.links['data'].values())
labels = np.zeros(
(dl.nodes['count'][0], dl.labels_train['num_labels']), dtype=float)
val_ratio = 0.2
train_idx = np.nonzero(dl.labels_train['mask'])[0]
np.random.shuffle(train_idx)
split = int(train_idx.shape[0]*val_ratio)
val_idx = train_idx[:split]
train_idx = train_idx[split:]
train_idx = np.sort(train_idx)
val_idx = np.sort(val_idx)
test_idx = np.nonzero(dl.labels_test['mask'])[0]
labels[train_idx] = dl.labels_train['data'][train_idx]
labels[val_idx] = dl.labels_train['data'][val_idx]
# labels[test_idx] = dl.labels_test['data'][test_idx]
# if prefix != 'IMDB':
#     labels = labels.argmax(axis=1)
train_val_test_idx = {}
train_val_test_idx['train_idx'] = train_idx
train_val_test_idx['val_idx'] = val_idx
train_val_test_idx['test_idx'] = test_idx

In [17]:
features[0].shape

(384, 1600, 1600)

In [19]:
features_list = features

In [21]:
def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

In [26]:
torch.from_numpy(features_list[0]).type(torch.FloatTensor).shape

torch.Size([384, 1600, 1600])

In [27]:
torch.from_numpy(features_list[1]).type(torch.FloatTensor).shape

torch.Size([616, 129])

In [32]:
import torch
import torch.nn as nn

In [30]:
features_list = [mat2tensor(features).to('cpu')
                     for features in features_list]

In [33]:
class Conv2dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(Conv2dBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))

In [34]:
# (out_channels, kernel_size, stride, padding)
enzyme_conv_archi = [
    # input: 1600x1600x1
    (1, 4, 4, 0),
    # 400x400x1
    (1, 4, 4, 0),
    # 100x100x1
    (1, 4, 4, 0),
    # 25x25x1
]

In [96]:
class GCN(nn.Module):
    def __init__(self,
                 g,
                 m_dim,
                 in_dims,
                 num_hidden,
                 num_classes,
                 num_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        self.layers = nn.ModuleList()
        
        # conv2d layers for enzyme nodes
        e_layers = []
        in_channels = 1
        for x in enzyme_conv_archi:
            if type(x) == tuple:
                e_layers += [
                    Conv2dBlock(
                        in_channels, x[0], kernel_size=x[1], stride=x[2], padding=x[3],
                    )
                ]
                in_channels = x[0]
                
        self.e_conv = nn.Sequential(*e_layers)

        # fc layers: to make the features of all the nodes become the same dimension  
        in_dims = [625, m_dim]
              
        self.fc_list = nn.ModuleList(
            [nn.Linear(in_dim, num_hidden, bias=True) for in_dim in in_dims])
        for fc in self.fc_list:
            nn.init.xavier_normal_(fc.weight, gain=1.414)

        # input layer
        self.layers.append(
            GraphConv(num_hidden, num_hidden, activation=activation))

        # hidden layers
        for i in range(num_layers - 1):
            self.layers.append(
                GraphConv(num_hidden, num_hidden, activation=activation))

        # output layer
        self.layers.append(GraphConv(num_hidden, num_classes))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, features_list):
        h = []
        
        e_feature = features_list[0].unsqueeze(1)
        e_feature = self.e_conv(e_feature).reshape((-1, 625))
        
        m_feature = features_list[1]
        
        features_list = [e_feature, m_feature]
        
        for fc, feature in zip(self.fc_list, features_list):
            h.append(fc(feature))
        h = torch.cat(h, 0)
        for i, layer in enumerate(self.layers):
            h = self.dropout(h)
            h = layer(self.g, h)
        return h

In [ ]:
# Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

In [97]:
net = GCN()
net

TypeError: __init__() missing 8 required positional arguments: 'g', 'm_dim', 'in_dims', 'num_hidden', 'num_classes', 'num_layers', 'activation', and 'dropout'

In [95]:
features_list[1].shape

torch.Size([616, 129])

In [92]:
result = net(features_list)

In [93]:
result.shape

torch.Size([384, 625])

In [85]:
result[0]

tensor([[[-3.6409e-01, -1.0104e-01,  5.0937e+00,  5.7468e+00,  4.5925e+00,
          -3.1782e-01, -2.5275e-01,  1.0278e+01,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02],
         [-3.3018e-01,  3.4949e+00,  7.1700e-02, -2.6454e-01, -5.7530e-01,
           4.0810e-01, -3.0830e-01,  1.1299e-01,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02],
         [ 1.6297e+00,  9.3666e-01, -4.9199e-01, -4.0784e-01,  2.1619e+00,
           1.5165e+00,  3.9424e+00, -7.8235e-03,  2.2031e-02,  2.2031e-02,
           2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
           2.2031e-02, 

In [89]:
r = result.reshape((-1, 625))
r[0]

tensor([-3.6409e-01, -1.0104e-01,  5.0937e+00,  5.7468e+00,  4.5925e+00,
        -3.1782e-01, -2.5275e-01,  1.0278e+01,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
        -3.3018e-01,  3.4949e+00,  7.1700e-02, -2.6454e-01, -5.7530e-01,
         4.0810e-01, -3.0830e-01,  1.1299e-01,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         1.6297e+00,  9.3666e-01, -4.9199e-01, -4.0784e-01,  2.1619e+00,
         1.5165e+00,  3.9424e+00, -7.8235e-03,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2031e-02,
         2.2031e-02,  2.2031e-02,  2.2031e-02,  2.2

In [54]:
features_list[0].shape

torch.Size([384, 1600, 1600])

In [56]:
features_list[0].unsqueeze(0).shape

torch.Size([1, 384, 1600, 1600])